In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch, sys
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from pathlib import Path
from tqdm import tqdm

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

# Set dataset path accordingly
if IN_COLAB:
    ! git clone https://github.com/MrKiwix/IAPR-project.git
    %cd IAPR-project
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT_DIR = Path('/content/drive/MyDrive/IAPR')
else:
    ROOT_DIR = Path('./')

# Training Notebook

Function calls to start a training.

This assumes the following:
- Training data is present at the desired location
- The CSV label information is also created

> Warning: this exports the best model weights and will therefore erase the previous one with the same name

We start with the constant and transformation setup:

In [ ]:
# Constants

SYNTHETIC_DATA = True
NUM_CLASSES = 13
IMG_SIZE = (200, 300) # (height, width)
BATCH_SIZE = 32
NUM_EPOCHS = 60

# Path to dataset and csv label

if SYNTHETIC_DATA:
    label_csv  = ROOT_DIR / Path("./data/synthetic_train.csv")
    images_dir = ROOT_DIR / Path("./data/synthetic_train")
else:
    label_csv  = ROOT_DIR / Path("./data/train.csv")
    images_dir = ROOT_DIR / Path("./data/train")
best_model_path = ROOT_DIR / Path("./model/best_choco_model.pt")
# Create the model directory if it doesn't exist
model_dir = best_model_path.parent
model_dir.mkdir(parents=True, exist_ok=True)

# Training and eval transform
train_tf = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True), 
    v2.Resize(IMG_SIZE, antialias=True),                
    v2.ColorJitter(0.2, 0.2, 0.2, 0.1), # Not needed in the eval transform
    v2.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

val_tf = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True), 
    v2.Resize(IMG_SIZE, antialias=True),                
    v2.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

Now, we can load our dataset to create a training and validation split

In [ ]:
from src.data.TrainChocolateDataset import * 

# We first create a general dataset
train_eval_dataset = ChocolateDataset(
    data_dir=images_dir,
    label_csv=label_csv,
    transform=None, # Since the two split are not using the same transform, we set it to None
    target_transform=LabelToTensor(),
)
# We now split the dataset into training and validation sets
# Split indexes
train_len = int(0.8 * len(train_eval_dataset))
test_len  = len(train_eval_dataset) - train_len
train_idxs, test_idxs = torch.utils.data.random_split(
    range(len(train_eval_dataset)), [train_len, test_len], generator=torch.Generator().manual_seed(42))


training_dataset = Subset(
    ChocolateDataset(images_dir, label_csv, transform=train_tf, target_transform=LabelToTensor()),
    train_idxs)
val_dataset = Subset(
    ChocolateDataset(images_dir, label_csv, transform=val_tf, target_transform=LabelToTensor()),
    test_idxs)

# Create DataLoaders
num_workers = 0
train_loader = DataLoader(training_dataset, BATCH_SIZE,
                          shuffle=True,  num_workers=num_workers, pin_memory=True)
val_loader  = DataLoader(val_dataset,  BATCH_SIZE,
                          shuffle=False, num_workers=num_workers, pin_memory=True)


Data is now ready with our loader, let's instantiate the model

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Import the model and the training settings
from src.model.ChocoNetwork import ChocoNetwork
from src.training.training import *

model = ChocoNetwork().to(device)
loss = get_loss() # Smooth L1 (Huber) with β=1.0
optimizer = get_optimizer(model) # AdamW with weight decay

best_val_loss = float('inf')

# Let's train this model
for epoch in tqdm(range(1, NUM_EPOCHS + 1)):
    
    train_loss = train_epoch(train_loader, model, loss, optimizer, device)
    val_loss, val_mae = eval_epoch(val_loader, model, loss, NUM_CLASSES, device)

    # ---- logging ----
    mae_str = ", ".join([f"{m:.2f}" for m in val_mae])
    print(f"Epoch {epoch:02d} | "
          f"train loss: {train_loss:.4f} | "
          f"val loss: {val_loss:.4f} | "
          f"val MAE/class: [{mae_str}]")

    # save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)



Using device: cuda


  0%|          | 0/60 [00:24<?, ?it/s]


KeyboardInterrupt: 